# Hyperpod Inference Admin Notebook

## 1.0 Prerequisistes
### 1.1 Permissions
<a id='section_1.1'></a>

This notebook is to be run by Administrator (with Administrator access) plus sagemaker service in the trust policy, and cluster admin access to EKS cluster

For access to EKS cluster

- Go to EKS console and select the cluster you are using.
  - Note: The correlated EKS cluster name is shown in the output of the above cell.
- Look in  the "Access" tab and select "IAM Access Entries"
- If there is not an entry for your execution role:
  - Select "Create Access Entry"
  - Select the desired execution role and correlate the `AmazonEKSClusterAdminPolicy` with the role

### 1.2 Permissions
- Ensure the role run this notebook has the Admin Access

## 2.0 Set up environment 

### 2.1 Set up environment variables

In [7]:
# Name of your EKS cluster
EKS_CLUSTER_NAME="modelhub10-eks"

# Region
REGION="us-east-1"

# Account Id
ACCOUNT_ID="434444145045"

# Name of Hyperpod cluster
HP_CLUSTER_NAME="modelhub10"

# ARN of Hyperpod cluster
HP_CLUSTER_ARN="arn:aws:sagemaker:us-east-1:434444145045:cluster/e5j2ja5nw13e"

# ID of HyperPod cluster (last 12 characters of ARN)
HP_CLUSTER_ID="e5j2ja5nw13e"

# S3 bucket where tls certificates will be uploaded
TLS_BUCKET_NAME="sagemaker-us-east-1-434444145045"

# VPC Id
VPC_ID="vpc-0d74c586a6ea0e2bd"

In [2]:
LB_CONTROLLER_POLICY_NAME = "LBControllerPolicy-" + HP_CLUSTER_ID
S3_MOUNT_ACCESS_POLICY_NAME = "S3MountpointAccessPolicy-" + HP_CLUSTER_ID
S3_CSI_ROLE_NAME = "S3CSIRole-" + HP_CLUSTER_ID
KEDA_OPERATOR_POLICY_NAME = "KedaOperatorPolicy-" + HP_CLUSTER_ID
KEDA_OPERATOR_ROLE_NAME = "KedaOperatorRole-" + HP_CLUSTER_ID
PRESIGNED_URL_ACCESS_POLICY_NAME = "PresignedUrlAccessPolicy" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_ACCESS_POLICY_NAME = "HyperpodInferenceAccessPolicy" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_ROLE_NAME = "HyperpodInferenceRole-" + HP_CLUSTER_ID
HYPERPOD_INFERENCE_SA_NAME="hyperpod-inference-operator-controller"
HYPERPOD_INFERENCE_SA_NAMESPACE="hyperpod-inference-system"
JUMPSTART_GATED_ROLE_NAME = "JumpstartGatedRole-" + HP_CLUSTER_ID
FSX_CSI_ROLE_NAME = "FSxCSIDriverFullAccess-" + HP_CLUSTER_ID

### 2.2 Install dependencies 

Following script downloads and installs Helm, Kubectl and Eksctl if not already installed.

In [3]:
!git clone https://github.com/aws/sagemaker-hyperpod-cli.git

Cloning into 'sagemaker-hyperpod-cli'...
remote: Enumerating objects: 7016, done.
remote: Counting objects: 100% (1822/1822), done.
remote: Compressing objects: 100% (711/711), done.
remote: Total 7016 (delta 1420), reused 1111 (delta 1111), pack-reused 5194 (from 3)
Receiving objects: 100% (7016/7016), 3.78 MiB | 36.91 MiB/s, done.
Resolving deltas: 100% (3776/3776), done.


In [4]:
!chmod +x sagemaker-hyperpod-cli/helm_chart/install_dependencies.sh
!./sagemaker-hyperpod-cli/helm_chart/install_dependencies.sh

helm is already installed. Version: version.BuildInfo{Version:"v4.0.1", GitCommit:"12500dd401faa7629f30ba5d5bff36287f3e94d3", GitTreeState:"clean", GoVersion:"go1.25.4", KubeClientVersion:"v1.34"}
kubectl is already installed. Version: Client Version: v1.34.2
Kustomize Version: v5.7.1
eksctl is already installed. Version: 0.220.0
Installation check completed successfully!
Installed versions:
helm: version.BuildInfo{Version:"v4.0.1", GitCommit:"12500dd401faa7629f30ba5d5bff36287f3e94d3", GitTreeState:"clean", GoVersion:"go1.25.4", KubeClientVersion:"v1.34"}
kubectl: Client Version: v1.34.2
Kustomize Version: v5.7.1
eksctl: 0.220.0
All dependencies installed successfully.


## 3.0 Connect to your EKS Cluster (via kubeconfig)

In [3]:
!aws eks update-kubeconfig --name "$EKS_CLUSTER_NAME" --region "$REGION" --output json

Added new context arn:aws:eks:us-east-1:434444145045:cluster/sagemaker-hyperpod-eks-cluster to /home/ubuntu/.kube/config


### 3.1 Confirm Successful Connection to Cluster

The below commands should not show any errors.

In [4]:
!aws sts get-caller-identity

{
    "UserId": "AROAWKJXDSGK5D7ZWT4CB:i-0097bd29342c267b7",
    "Account": "434444145045",
    "Arn": "arn:aws:sts::434444145045:assumed-role/admin_role_for_workshop/i-0097bd29342c267b7"
}


# create-access-entry

For access to EKS cluster with the role above

Go to EKS console and select the cluster you are using.
Note: The correlated EKS cluster name is shown in the output of the above cell.
Look in the "Access" tab and select "IAM Access Entries"
If there is not an entry for your execution role:
Select "Create Access Entry"
Select the desired execution role and correlate the AmazonEKSClusterAdminPolicy with the role

In [6]:
!kubectl config current-context
!kubectl get svc

arn:aws:eks:us-east-1:434444145045:cluster/sagemaker-hyperpod-eks-cluster


E1215 07:06:35.941020 2432653 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: the server has asked for the client to provide credentials"
E1215 07:06:36.662675 2432653 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: the server has asked for the client to provide credentials"
E1215 07:06:37.391286 2432653 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: the server has asked for the client to provide credentials"
E1215 07:06:38.123626 2432653 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: the server has asked for the client to provide credentials"
E1215 07:06:38.840235 2432653 memcache.go:265] "Unhandled Error" err="couldn't get current server API group list: the server has asked for the client to provide credentials"
error: You must be logged in to the server (the server has asked for the client to provide credentials)


### 3.2 Associate IAM OIDC Provider with EKS Cluster

In [78]:
!eksctl utils associate-iam-oidc-provider --cluster $EKS_CLUSTER_NAME --region $REGION --approve

]11;?\

2025-12-14 05:56:54 [ℹ]  IAM Open ID Connect provider is already associated with cluster "modelhub3-eks" in "us-east-1"


## 4.0 Load balancer controller installation

### 4.1 Create IAM Policy for AWSLoadBalancer

In [79]:
!curl -o /tmp/AWSLoadBalancerControllerIAMPolicy.json https://raw.githubusercontent.com/kubernetes-sigs/aws-load-balancer-controller/v2.13.0/docs/install/iam_policy.json
!aws iam create-policy --policy-name $LB_CONTROLLER_POLICY_NAME --policy-document file:///tmp/AWSLoadBalancerControllerIAMPolicy.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8955  100  8955    0     0   200k      0 --:--:-- --:--:-- --:--:--  203k



An error occurred (EntityAlreadyExists) when calling the CreatePolicy operation: A policy called LBControllerPolicy-tru6trz178yz already exists. Duplicate names are not allowed.


### 4.2 Create the AWSLoadBalancer Service Account

In [51]:
query = "'Policies[?PolicyName==`" + LB_CONTROLLER_POLICY_NAME + "`].Arn'"
policy_arn_list=!(aws iam list-policies --query $query --output text)

policy_arn=policy_arn_list[0]

!eksctl create iamserviceaccount \
    --approve \
    --override-existing-serviceaccounts \
    --name= "aws-load-balancer-controller" \
    --namespace=kube-system \
    --cluster=$EKS_CLUSTER_NAME \
    --attach-policy-arn=$policy_arn \
    --region $REGION

]11;?\

2025-12-14 05:51:48 [ℹ]  1 existing iamserviceaccount(s) (kube-system/aws-load-balancer-controller) will be excluded
2025-12-14 05:51:48 [ℹ]  1 iamserviceaccount (kube-system/aws-load-balancer-controller) was excluded (based on the include/exclude rules)
2025-12-14 05:51:48 [!]  metadata of serviceaccounts that exist in Kubernetes will be updated, as --override-existing-serviceaccounts was set
2025-12-14 05:51:48 [ℹ]  no tasks


### 4.3 Apply Tags to all Subnets in your EKS Cluster (public and private)

In [52]:
filters = "'[{\"Name\":\"vpc-id\", \"Values\": [\"" + VPC_ID + "\"]}, {\"Name\":\"map-public-ip-on-launch\", \"Values\":[\"true\"]}]'"
subnets = !(aws ec2 describe-subnets --filters $filters --output json | jq '.Subnets[]' | jq --raw-output '.SubnetId')
!echo "Applied tags to following subnets"
!echo $subnets
for subnet in subnets:
    !(aws ec2 create-tags --resources $subnet --tags Key=kubernetes.io/role/elb,Value=1)

Applied tags to following subnets


[]


## 5.0 Namespace creation


### 5.1 KEDA Controller Namespace Creation

In [53]:
!kubectl create namespace keda

Error from server (AlreadyExists): namespaces "keda" already exists


### 5.2 Cert Manager Namespace Creation

In [55]:
!kubectl create namespace cert-manager

Error from server (AlreadyExists): namespaces "cert-manager" already exists


### 5.3 Endpoint creation

In [56]:
from IPython.utils.capture import capture_output

with capture_output() as c:
    !(aws ec2 describe-route-tables --region $REGION --filters "Name=vpc-id,Values=$VPC_ID" --query 'RouteTables[].Associations[].RouteTableId' | jq 'unique' | jq -r 'join (" ")')
ROUTE_TABLE_IDS = c.stdout.strip()

SERVICE_NAME="com.amazonaws." + REGION + ".s3"

In [57]:
# If vpc endpoint is already created via cloudformation, then remove it
import time
import subprocess
from IPython.utils.capture import capture_output

# Check for existing endpoint with corrected query
with capture_output() as c:
    !(aws ec2 describe-vpc-endpoints \
        --region "$REGION" \
        --filters "Name=vpc-id,Values=$VPC_ID" "Name=service-name,Values=$SERVICE_NAME" \
        --query 'VpcEndpoints[?State!=`deleted`].[VpcEndpointId]' \
        --output text)
EXISTING_ENDPOINT = c.stdout.strip()

# If endpoint exists and is not empty/None and only delete S3 VPC endpoint , delete it
if EXISTING_ENDPOINT and EXISTING_ENDPOINT != "None" and "s3" in SERVICE_NAME.lower():
    print("Found existing endpoint, showing details:")
    !(aws ec2 describe-vpc-endpoints --vpc-endpoint-ids "$EXISTING_ENDPOINT" --region "$REGION")

    # Delete the VPC endpoint
    !aws ec2 delete-vpc-endpoints --vpc-endpoint-ids {EXISTING_ENDPOINT} --region {REGION}

    # Wait until the endpoint is fully deleted
    print("Waiting for VPC endpoint to be deleted...")
    SLEEP_INTERVAL = 5
    while True:
        result = subprocess.run(
            ["aws", "ec2", "describe-vpc-endpoints", "--vpc-endpoint-ids", EXISTING_ENDPOINT, "--region", REGION],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
        if result.returncode != 0:
            break
        print(".", end="", flush=True)
        time.sleep(SLEEP_INTERVAL)
    print(f"VPC endpoint '{EXISTING_ENDPOINT}' has been successfully deleted.")
else:
    print("No existing endpoint found.")

Found existing endpoint, showing details:
{
    "VpcEndpoints": [
        {
            "VpcEndpointId": "vpce-06492b88463b5e629",
            "VpcEndpointType": "Gateway",
            "VpcId": "vpc-0363d0a8514bd0587",
            "ServiceName": "com.amazonaws.us-east-1.s3",
            "State": "available",
            "PolicyDocument": "{\"Version\":\"2008-10-17\",\"Statement\":[{\"Effect\":\"Allow\",\"Principal\":\"*\",\"Action\":\"*\",\"Resource\":\"*\"}]}",
            "RouteTableIds": [
                "rtb-00bafb29d5edf8ab9",
                "rtb-061e2c1b21bf5cb51"
            ],
            "SubnetIds": [],
            "Groups": [],
            "IpAddressType": "ipv4",
            "DnsOptions": {
                "DnsRecordIpType": "service-defined"
            },
            "PrivateDnsEnabled": false,
            "RequesterManaged": false,
            "NetworkInterfaceIds": [],
            "DnsEntries": [],
            "CreationTimestamp": "2025-12-14T05:26:31+00:00",
        

In [58]:
!(aws ec2 create-vpc-endpoint --vpc-id $VPC_ID --vpc-endpoint-type "Gateway" --service-name $SERVICE_NAME --route-table-ids $ROUTE_TABLE_IDS --region $REGION)

{
    "VpcEndpoint": {
        "VpcEndpointId": "vpce-0e2aad4dabe616fab",
        "VpcEndpointType": "Gateway",
        "VpcId": "vpc-0363d0a8514bd0587",
        "ServiceName": "com.amazonaws.us-east-1.s3",
        "State": "available",
        "PolicyDocument": "{\"Version\":\"2008-10-17\",\"Statement\":[{\"Effect\":\"Allow\",\"Principal\":\"*\",\"Action\":\"*\",\"Resource\":\"*\"}]}",
        "RouteTableIds": [
            "rtb-00bafb29d5edf8ab9",
            "rtb-0c7a91e6e62ed53ca",
            "rtb-061e2c1b21bf5cb51"
        ],
        "SubnetIds": [],
        "Groups": [],
        "IpAddressType": "ipv4",
        "DnsOptions": {
            "DnsRecordIpType": "service-defined"
        },
        "PrivateDnsEnabled": false,
        "RequesterManaged": false,
        "NetworkInterfaceIds": [],
        "DnsEntries": [],
        "CreationTimestamp": "2025-12-14T05:52:53+00:00",
        "OwnerId": "434444145045"
    }
}


## 6.0 S3 CSI driver installation

Setup Role for S3 Mountpoint

In [59]:
%%bash -s "$S3_MOUNT_ACCESS_POLICY_NAME"

cat <<EOF> /tmp/s3-role-policy.json
{
   "Version": "2012-10-17",
   "Statement": [
        {
            "Sid": "MountpointFullBucketAccess",
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                    "arn:aws:s3:::*",
                    "arn:aws:s3:::*/*"
            ]
        },
        {
            "Sid": "MountpointFullObjectAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:AbortMultipartUpload",
                "s3:DeleteObject"
            ],
            "Resource": [
                    "arn:aws:s3:::*",
                    "arn:aws:s3:::*/*"
            ]
        }
   ]
}
EOF

aws iam create-policy --policy-name $1 --policy-document file:///tmp/s3-role-policy.json


{
    "Policy": {
        "PolicyName": "S3MountpointAccessPolicy-tru6trz178yz",
        "PolicyId": "ANPAWKJXDSGK5LDHEEQQT",
        "Arn": "arn:aws:iam::434444145045:policy/S3MountpointAccessPolicy-tru6trz178yz",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2025-12-14T05:52:57+00:00",
        "UpdateDate": "2025-12-14T05:52:57+00:00"
    }
}


In [60]:
query = "'Policies[?PolicyName==`" + S3_MOUNT_ACCESS_POLICY_NAME + "`].Arn'"
S3_MOUNT_POLICY_ARN_LIST=!(aws iam list-policies --query $query)
S3_MOUNT_POLICY_ARN = S3_MOUNT_POLICY_ARN_LIST[1].strip()

print(S3_MOUNT_POLICY_ARN)

"arn:aws:iam::434444145045:policy/S3MountpointAccessPolicy-tru6trz178yz"


In [61]:
!eksctl create iamserviceaccount \
    --name s3-csi-driver-sa \
    --override-existing-serviceaccounts \
    --namespace kube-system \
    --cluster $EKS_CLUSTER_NAME \
    --attach-policy-arn $S3_MOUNT_POLICY_ARN \
    --approve \
    --role-name $S3_CSI_ROLE_NAME \
    --region $REGION

]11;?\

2025-12-14 05:53:14 [ℹ]  1 existing iamserviceaccount(s) (kube-system/aws-load-balancer-controller) will be excluded
2025-12-14 05:53:14 [ℹ]  1 iamserviceaccount (kube-system/s3-csi-driver-sa) was included (based on the include/exclude rules)
2025-12-14 05:53:14 [!]  metadata of serviceaccounts that exist in Kubernetes will be updated, as --override-existing-serviceaccounts was set
2025-12-14 05:53:14 [ℹ]  1 task: { 
    2 sequential sub-tasks: { 
        create IAM role for serviceaccount "kube-system/s3-csi-driver-sa",
        create serviceaccount "kube-system/s3-csi-driver-sa",
    } }2025-12-14 05:53:14 [ℹ]  building iamserviceaccount stack "eksctl-modelhub3-eks-addon-iamserviceaccount-kube-system-s3-csi-driver-sa"
2025-12-14 05:53:15 [ℹ]  deploying stack "eksctl-modelhub3-eks-addon-iamserviceaccount-kube-system-s3-csi-driver-sa"
2025-12-14 05:53:15 [ℹ]  waiting for CloudFormation stack "eksctl-modelhub3-eks-addon-iamserviceaccount-kube-system-s3-csi-driver-sa"
2025-12-14 05:53:45

In [62]:
!kubectl label serviceaccount s3-csi-driver-sa app.kubernetes.io/component=csi-driver app.kubernetes.io/instance=aws-mountpoint-s3-csi-driver app.kubernetes.io/managed-by=EKS app.kubernetes.io/name=aws-mountpoint-s3-csi-driver -n kube-system --overwrite

serviceaccount/s3-csi-driver-sa labeled


## 6.0.1 keda operator role Creation

In [63]:
OIDC_ID = !aws eks describe-cluster --name $EKS_CLUSTER_NAME --query "cluster.identity.oidc.issuer" --output text | cut -d '/' -f 5
OIDC_ID = OIDC_ID[0]

In [80]:
%%bash -s "$REGION" "$OIDC_ID" "$ACCOUNT_ID" "$KEDA_OPERATOR_POLICY_NAME" "$KEDA_OPERATOR_ROLE_NAME"

REGION=$1
OIDC_ID=$2
ACCOUNT_ID=$3
KEDA_OPERATOR_POLICY_NAME=$4
KEDA_OPERATOR_ROLE_NAME=$5

# Create trust policy
cat <<EOF > /tmp/keda-trust-policy.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:kube-system:keda-operator",
                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"
                }
            }
        }
    ]
}
EOF
 
# Create permissions policy
cat <<EOF > /tmp/keda-policy.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:GetMetricData",
                "cloudwatch:GetMetricStatistics",
                "cloudwatch:ListMetrics"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "aps:QueryMetrics",
                "aps:GetLabels",
                "aps:GetSeries",
                "aps:GetMetricMetadata"
            ],
            "Resource": "*"
        }
    ]
}
EOF
 
# Create the role
aws iam create-role \
    --role-name $KEDA_OPERATOR_ROLE_NAME \
    --assume-role-policy-document file:///tmp/keda-trust-policy.json
 
# Create the policy
POLICY_ARN=$(aws iam create-policy \
    --policy-name $KEDA_OPERATOR_POLICY_NAME \
    --policy-document file:///tmp/keda-policy.json \
    --query 'Policy.Arn' \
    --output text)
 
# Attach the policy to the role
aws iam attach-role-policy \
    --role-name $KEDA_OPERATOR_ROLE_NAME \
    --policy-arn $POLICY_ARN


An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name KedaOperatorRole-tru6trz178yz already exists.

An error occurred (EntityAlreadyExists) when calling the CreatePolicy operation: A policy called KedaOperatorPolicy-tru6trz178yz already exists. Duplicate names are not allowed.

aws: [ERROR]: argument --policy-arn: expected one argument

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help



CalledProcessError: Command 'b'\nREGION=$1\nOIDC_ID=$2\nACCOUNT_ID=$3\nKEDA_OPERATOR_POLICY_NAME=$4\nKEDA_OPERATOR_ROLE_NAME=$5\n\n# Create trust policy\ncat <<EOF > /tmp/keda-trust-policy.json\n{\n    "Version": "2012-10-17",\n    "Statement": [\n        {\n            "Effect": "Allow",\n            "Principal": {\n                "Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"\n            },\n            "Action": "sts:AssumeRoleWithWebIdentity",\n            "Condition": {\n                "StringLike": {\n                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:kube-system:keda-operator",\n                    "oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"\n                }\n            }\n        }\n    ]\n}\nEOF\n\n# Create permissions policy\ncat <<EOF > /tmp/keda-policy.json\n{\n    "Version": "2012-10-17",\n    "Statement": [\n        {\n            "Effect": "Allow",\

In [23]:
%%bash -s $REGION $PRESIGNED_URL_ACCESS_POLICY_NAME

cat <<EOF> /tmp/presignedurl-policy.json
{
   "Version": "2012-10-17",
   "Statement": [
        {
            "Sid": "CreatePresignedUrlAccess",
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateHubContentPresignedUrls"
            ],
            "Resource": [
                "arn:aws:sagemaker:$1:aws:hub/SageMakerPublicHub", 
                "arn:aws:sagemaker:$1:aws:hub-content/SageMakerPublicHub/*/*" 
            ]
        }
   ]
}
EOF

aws iam create-policy --policy-name "$2" --policy-document file:///tmp/presignedurl-policy.json

{
    "Policy": {
        "PolicyName": "PresignedUrlAccessPolicyqwz5zh2q56c1",
        "PolicyId": "ANPAWKJXDSGKXTV47JY5L",
        "Arn": "arn:aws:iam::434444145045:policy/PresignedUrlAccessPolicyqwz5zh2q56c1",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2025-12-13T12:52:19+00:00",
        "UpdateDate": "2025-12-13T12:52:19+00:00"
    }
}


## 7.0 Create execution role

#### 7.1.2 Import the Hyperpod Inference Access Policy to IAM

In [ ]:
# !aws iam create-policy --policy-name $HYPERPOD_INFERENCE_ACCESS_POLICY_NAME --policy-document file://inference-resources/hyperpod_inference_operator_policy.json


Error parsing parameter '--policy-document': Unable to load paramfile file://inference-resources/hyperpod_inference_operator_policy.json: [Errno 2] No such file or directory: 'inference-resources/hyperpod_inference_operator_policy.json'


In [81]:
%%bash -s $REGION $HYPERPOD_INFERENCE_ACCESS_POLICY_NAME

cat << EOF > /tmp/permission-policy.json
{
    "Version": "2012-10-17",		 	 	 
    "Statement": [
        {
        "Sid": "S3Access",
        "Effect": "Allow",
        "Action": [
            "s3:Get*",
            "s3:List*",
            "s3:Describe*",
            "s3:PutObject"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "ECRAccess",
        "Effect": "Allow",
        "Action": [
            "ecr:GetAuthorizationToken",
            "ecr:BatchCheckLayerAvailability",
            "ecr:GetDownloadUrlForLayer",
            "ecr:GetRepositoryPolicy",
            "ecr:DescribeRepositories",
            "ecr:ListImages",
            "ecr:DescribeImages",
            "ecr:BatchGetImage",
            "ecr:GetLifecyclePolicy",
            "ecr:GetLifecyclePolicyPreview",
            "ecr:ListTagsForResource",
            "ecr:DescribeImageScanFindings"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "EC2Access",
        "Effect": "Allow",
        "Action": [
            "ec2:AssignPrivateIpAddresses",
            "ec2:AttachNetworkInterface",
            "ec2:CreateNetworkInterface",
            "ec2:DeleteNetworkInterface",
            "ec2:DescribeInstances",
            "ec2:DescribeTags",
            "ec2:DescribeNetworkInterfaces",
            "ec2:DescribeInstanceTypes",
            "ec2:DescribeSubnets",
            "ec2:DetachNetworkInterface",
            "ec2:ModifyNetworkInterfaceAttribute",
            "ec2:UnassignPrivateIpAddresses",
            "ec2:CreateTags",
            "ec2:DescribeInstances",
            "ec2:DescribeInstanceTypes",
            "ec2:DescribeRouteTables",
            "ec2:DescribeSecurityGroups",
            "ec2:DescribeSubnets",
            "ec2:DescribeVolumes",
            "ec2:DescribeVolumesModifications",
            "ec2:DescribeVpcs",
            "ec2:CreateVpcEndpointServiceConfiguration",
            "ec2:DeleteVpcEndpointServiceConfigurations",
            "ec2:DescribeVpcEndpointServiceConfigurations",
            "ec2:ModifyVpcEndpointServicePermissions"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "EKSAuthAccess",
        "Effect": "Allow",
        "Action": [
            "eks-auth:AssumeRoleForPodIdentity"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "EKSAccess",
        "Effect": "Allow",
        "Action": [
            "eks:AssociateAccessPolicy",
            "eks:Describe*",
            "eks:List*",
            "eks:AccessKubernetesApi"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "ApiGatewayAccess",
        "Effect": "Allow",
        "Action": [
            "apigateway:POST",
            "apigateway:GET",
            "apigateway:PUT",
            "apigateway:PATCH",
            "apigateway:DELETE",
            "apigateway:UpdateRestApiPolicy"
        ],
        "Resource": [
            "arn:aws:apigateway:*::/vpclinks",
            "arn:aws:apigateway:*::/vpclinks/*",
            "arn:aws:apigateway:*::/restapis",
            "arn:aws:apigateway:*::/restapis/*"
        ]
        },
        {
        "Sid": "ElasticLoadBalancingAccess",
        "Effect": "Allow",
        "Action": [
            "elasticloadbalancing:CreateLoadBalancer",
            "elasticloadbalancing:DescribeLoadBalancers",
            "elasticloadbalancing:DescribeLoadBalancerAttributes",
            "elasticloadbalancing:DescribeListeners",
            "elasticloadbalancing:DescribeListenerCertificates",
            "elasticloadbalancing:DescribeSSLPolicies",
            "elasticloadbalancing:DescribeRules",
            "elasticloadbalancing:DescribeTargetGroups",
            "elasticloadbalancing:DescribeTargetGroupAttributes",
            "elasticloadbalancing:DescribeTargetHealth",
            "elasticloadbalancing:DescribeTags",
            "elasticloadbalancing:DescribeTrustStores",
            "elasticloadbalancing:DescribeListenerAttributes"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "SageMakerAccess",
        "Effect": "Allow",
        "Action": [
            "sagemaker:*"
        ],
        "Resource": [
            "*"
        ]
        },
        {
        "Sid": "AllowPassRoleToSageMaker",
        "Effect": "Allow",
        "Action": [
            "iam:PassRole"
        ],
        "Resource": "arn:aws:iam::*:role/*",
        "Condition": {
            "StringEquals": {
            "iam:PassedToService": "sagemaker.amazonaws.com"
            }
        }
        },
        {
        "Sid": "AcmAccess",
        "Effect": "Allow",
        "Action": [
            "acm:ImportCertificate",
            "acm:DeleteCertificate"
        ],
        "Resource": [
            "*"
        ]
        }
    ]
}
EOF

#### 7.1.3 Create an IAM role with the Hyperpod Inference Access Policy

In [67]:
!aws iam create-policy --policy-name $HYPERPOD_INFERENCE_ACCESS_POLICY_NAME --policy-document file:///tmp/permission-policy.json

{
    "Policy": {
        "PolicyName": "HyperpodInferenceAccessPolicytru6trz178yz",
        "PolicyId": "ANPAWKJXDSGK6UMBQBZ56",
        "Arn": "arn:aws:iam::434444145045:policy/HyperpodInferenceAccessPolicytru6trz178yz",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2025-12-14T05:54:16+00:00",
        "UpdateDate": "2025-12-14T05:54:16+00:00"
    }
}


In [68]:
query = "'Policies[?PolicyName==`" + HYPERPOD_INFERENCE_ACCESS_POLICY_NAME + "`].Arn'"
policy_arn_list=!(aws iam list-policies --query $query --output text)
policy_arn=policy_arn_list[0]

# Create the IAM role
!eksctl create iamserviceaccount --approve --role-only --name=$HYPERPOD_INFERENCE_SA_NAME --namespace=$HYPERPOD_INFERENCE_SA_NAMESPACE --cluster=$EKS_CLUSTER_NAME --attach-policy-arn=$policy_arn --role-name=$HYPERPOD_INFERENCE_ROLE_NAME --region=$REGION

]11;?\2025-12-14 05:54:26 [ℹ]  2 existing iamserviceaccount(s) (kube-system/aws-load-balancer-controller,kube-system/s3-csi-driver-sa) will be excluded
2025-12-14 05:54:26 [ℹ]  1 iamserviceaccount (hyperpod-inference-system/hyperpod-inference-operator-controller) was included (based on the include/exclude rules)
2025-12-14 05:54:26 [!]  serviceaccounts in Kubernetes will not be created or modified, since the option --role-only is used
2025-12-14 05:54:26 [ℹ]  1 task: { create IAM role for serviceaccount "hyperpod-inference-system/hyperpod-inference-operator-controller" }
2025-12-14 05:54:26 [ℹ]  building iamserviceaccount stack "eksctl-modelhub3-eks-addon-iamserviceaccount-hyperpod-inference-system-hyperpod-inference-operator-controller"
2025-12-14 05:54:26 [ℹ]  deploying stack "eksctl-modelhub3-eks-addon-iamserviceaccount-hyperpod-inference-system-hyperpod-inference-operator-controller"
2025-12-14 05:54:26 [ℹ]  waiting for CloudFormation stack "eksctl-modelhub3-eks-addon-iamservicea

### 7.2 Presigned URL support

#### 7.2.1 Create role for jumpstart gated model (Optional only for jumpstart gated model)

In [70]:
JUMPSTART_GATED_ROLE_NAME=f"JumpstartGatedRole-{REGION}-{HP_CLUSTER_NAME}"

In [82]:
%%bash -s "$REGION" "$OIDC_ID" "$ACCOUNT_ID" "$JUMPSTART_GATED_ROLE_NAME" "$PRESIGNED_URL_ACCESS_POLICY_NAME"

REGION=$1
OIDC_ID=$2
ACCOUNT_ID=$3
JUMPSTART_GATED_ROLE_NAME=$4
PRESIGNED_URL_ACCESS_POLICY_NAME=$5

# Create trust policy
cat <<EOF > /tmp/trust-policy.json
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Effect": "Allow",
			"Principal": {
				"Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"
			},
			"Action": "sts:AssumeRoleWithWebIdentity",
			"Condition": {
				"StringLike": {
					"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:*:hyperpod-inference-service-account",
					"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"
				}
			}
		}
	]
}
EOF

# Create the role using existing trust policy
aws iam create-role \
    --role-name $JUMPSTART_GATED_ROLE_NAME \
    --assume-role-policy-document file:///tmp/trust-policy.json

# Attach the existing PresignedUrlAccessPolicy to the role
aws iam attach-role-policy \
    --role-name $JUMPSTART_GATED_ROLE_NAME \
    --policy-arn arn:aws:iam::${ACCOUNT_ID}:policy/$PRESIGNED_URL_ACCESS_POLICY_NAME


An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name JumpstartGatedRole-tru6trz178yz already exists.

An error occurred (NoSuchEntity) when calling the AttachRolePolicy operation: Policy arn:aws:iam::434444145045:policy/PresignedUrlAccessPolicytru6trz178yz does not exist or is not attachable.


CalledProcessError: Command 'b'\nREGION=$1\nOIDC_ID=$2\nACCOUNT_ID=$3\nJUMPSTART_GATED_ROLE_NAME=$4\nPRESIGNED_URL_ACCESS_POLICY_NAME=$5\n\n# Create trust policy\ncat <<EOF > /tmp/trust-policy.json\n{\n\t"Version": "2012-10-17",\n\t"Statement": [\n\t\t{\n\t\t\t"Effect": "Allow",\n\t\t\t"Principal": {\n\t\t\t\t"Federated": "arn:aws:iam::$ACCOUNT_ID:oidc-provider/oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID"\n\t\t\t},\n\t\t\t"Action": "sts:AssumeRoleWithWebIdentity",\n\t\t\t"Condition": {\n\t\t\t\t"StringLike": {\n\t\t\t\t\t"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:sub": "system:serviceaccount:*:hyperpod-inference-service-account",\n\t\t\t\t\t"oidc.eks.$REGION.amazonaws.com/id/$OIDC_ID:aud": "sts.amazonaws.com"\n\t\t\t\t}\n\t\t\t}\n\t\t}\n\t]\n}\nEOF\n\n# Create the role using existing trust policy\naws iam create-role \\\n    --role-name $JUMPSTART_GATED_ROLE_NAME \\\n    --assume-role-policy-document file:///tmp/trust-policy.json\n\n# Attach the existing PresignedUrlAccessPolicy to the

In [83]:
JUMPSTART_GATED_ROLE_ARN_LIST= !aws iam get-role --role-name=$JUMPSTART_GATED_ROLE_NAME --query "Role.Arn" --output text
JUMPSTART_GATED_ROLE_ARN = JUMPSTART_GATED_ROLE_ARN_LIST[0]
!echo $JUMPSTART_GATED_ROLE_ARN

arn:aws:iam::434444145045:role/JumpstartGatedRole-tru6trz178yz


#### 7.2.2 Update the Trust Policy

In [84]:
%%bash -s "$HYPERPOD_INFERENCE_ROLE_NAME" "$HYPERPOD_INFERENCE_SA_NAMESPACE" "$HYPERPOD_INFERENCE_SA_NAME"

cat <<EOF> hyperpod_inference_additional_trust_statement.txt
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
EOF

aws iam get-role --role-name=$1 --query "Role"."AssumeRolePolicyDocument" > hyperpod_inference_trust_policy.json
if [[ $(diff hyperpod_inference_additional_trust_statement.txt hyperpod_inference_trust_policy.json | grep '^<' -) ]]; then
    echo Trust policy missing some permissions. Updating with the permissions above.
    { head -n3 hyperpod_inference_trust_policy.json && cat hyperpod_inference_additional_trust_statement.txt && tail -n+4 hyperpod_inference_trust_policy.json; } > /tmp/updated_hyperpod_inference_trust_policy.json
    # Also allow any service account to AssumeRoleWithWebIdentity
    sed -i '0,/"StringEquals": {/{s//"StringLike": {/}' /tmp/updated_hyperpod_inference_trust_policy.json
    sed -i "0,/:sub\": \"system:serviceaccount:${2}:${3}\"/{s//:sub\": \"system:serviceaccount:*:*\"/}" /tmp/updated_hyperpod_inference_trust_policy.json
    aws iam update-assume-role-policy --role-name=$1 --policy-document=file:///tmp/updated_hyperpod_inference_trust_policy.json
else
    echo Trust policy contains all necessary permissions.
fi

Trust policy missing some permissions. Updating with the permissions above.


### 7.4 Get role arns to be used during installation of operator

In [85]:
HYPERPOD_INFERENCE_ROLE_ARN_list= !(aws iam get-role --role-name=$HYPERPOD_INFERENCE_ROLE_NAME --query "Role.Arn" --output text)
HYPERPOD_INFERENCE_ROLE_ARN = HYPERPOD_INFERENCE_ROLE_ARN_list[0]
!echo $HYPERPOD_INFERENCE_ROLE_ARN

arn:aws:iam::434444145045:role/HyperpodInferenceRole-tru6trz178yz


In [86]:
S3_CSI_ROLE_ARN_LIST= !aws iam get-role --role-name=$S3_CSI_ROLE_NAME --query "Role.Arn" --output text
S3_CSI_ROLE_ARN = S3_CSI_ROLE_ARN_LIST[0]
!echo $S3_CSI_ROLE_ARN

arn:aws:iam::434444145045:role/S3CSIRole-tru6trz178yz


## 8.0 Install operator

In [ ]:
# HELM_CMD="helm upgrade --install hyperpod-dependencies ./sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator \
#      -n kube-system \
#      --set cert-manager.enabled=false \
#      --set inferenceOperators.enabled=true \
#      --set s3.enabled=false \
#      --set region=" + REGION + " \
#      --set eksClusterName=" + EKS_CLUSTER_NAME + " \
#      --set hyperpodClusterArn=" + HP_CLUSTER_ARN + " \
#      --set executionRoleArn=" + HYPERPOD_INFERENCE_ROLE_ARN + " \
#      --set s3.serviceAccountRoleArn=" + S3_CSI_ROLE_ARN + " \
#      --set s3.node.serviceAccount.create=false \
#      --set keda.podIdentity.aws.irsa.roleArn=\"arn:aws:iam::" + ACCOUNT_ID + ":role/keda-operator-role\" \
#      --set tlsCertificateS3Bucket=" + TLS_BUCKET_NAME + " \
#      --set alb.region=" + REGION + " \
#      --set alb.clusterName=" + EKS_CLUSTER_NAME + " \
#      --set alb.vpcId=" + VPC_ID + " \
#      --set jumpstartGatedModelDownloadRoleArn=" + JUMPSTART_GATED_ROLE_ARN


In [87]:
HELM_CMD="helm install hyperpod-inference-operator ./sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator \
     -n kube-system \
     --set region=" + REGION + " \
     --set eksClusterName=" + EKS_CLUSTER_NAME + " \
     --set hyperpodClusterArn=" + HP_CLUSTER_ARN + " \
     --set executionRoleArn=" + HYPERPOD_INFERENCE_ROLE_ARN + " \
     --set s3.serviceAccountRoleArn=" + S3_CSI_ROLE_ARN + " \
     --set s3.node.serviceAccount.create=false \
     --set keda.podIdentity.aws.irsa.roleArn=\"arn:aws:iam::" + ACCOUNT_ID + ":role/keda-operator-role\" \
     --set tlsCertificateS3Bucket=" + TLS_BUCKET_NAME + " \
     --set alb.region=" + REGION + " \
     --set alb.clusterName=" + EKS_CLUSTER_NAME + " \
     --set alb.vpcId=" + VPC_ID + " \
     --set jumpstartGatedModelDownloadRoleArn=" + JUMPSTART_GATED_ROLE_ARN

In [88]:
!echo $HELM_CMD

helm install hyperpod-inference-operator ./sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator -n kube-system --set region=us-east-1 --set eksClusterName=modelhub3-eks --set hyperpodClusterArn=arn:aws:sagemaker:us-east-1:434444145045:cluster/tru6trz178yz --set executionRoleArn=arn:aws:iam::434444145045:role/HyperpodInferenceRole-tru6trz178yz --set s3.serviceAccountRoleArn=arn:aws:iam::434444145045:role/S3CSIRole-tru6trz178yz --set s3.node.serviceAccount.create=false --set keda.podIdentity.aws.irsa.roleArn=arn:aws:iam::434444145045:role/keda-operator-role --set tlsCertificateS3Bucket=sagemaker-us-east-1-434444145045 --set alb.region=us-east-1 --set alb.clusterName=modelhub3-eks --set alb.vpcId=vpc-0363d0a8514bd0587 --set jumpstartGatedModelDownloadRoleArn=arn:aws:iam::434444145045:role/JumpstartGatedRole-tru6trz178yz


In [89]:
!helm dependencies update sagemaker-hyperpod-cli/helm_chart/HyperPodHelmChart/charts/inference-operator

Getting updates for unmanaged Helm repositories...
...Successfully got an update from the "https://kubernetes-sigs.github.io/metrics-server/" chart repository
...Successfully got an update from the "https://kubernetes-sigs.github.io/aws-fsx-csi-driver" chart repository
...Successfully got an update from the "https://awslabs.github.io/mountpoint-s3-csi-driver/" chart repository
...Successfully got an update from the "https://kedacore.github.io/charts" chart repository


...Successfully got an update from the "https://aws.github.io/eks-charts" chart repository
...Successfully got an update from the "https://charts.jetstack.io" chart repository
Saving 6 charts
Deleting outdated charts


In [90]:
!eval $HELM_CMD

Error: INSTALLATION FAILED: unable to continue with install: CustomResourceDefinition "certificaterequests.cert-manager.io" in namespace "" exists and cannot be imported into the current release: invalid ownership metadata; label validation error: missing key "app.kubernetes.io/managed-by": must be set to "Helm"; annotation validation error: missing key "meta.helm.sh/release-name": must be set to "hyperpod-inference-operator"; annotation validation error: missing key "meta.helm.sh/release-namespace": must be set to "kube-system"


## 9.0 FSx CSI Service Account Update

In [ ]:
 !eksctl create iamserviceaccount \
       --name fsx-csi-controller-sa \
       --override-existing-serviceaccounts \
       --namespace kube-system \
       --cluster $EKS_CLUSTER_NAME \
       --attach-policy-arn arn:aws:iam::aws:policy/AmazonFSxFullAccess \
       --role-name $FSX_CSI_ROLE_NAME \
       --approve \
       --region $REGION 

## 9.1 Deploy Model via UI Console

- Find out SageMaker Studio Role for Data Scientist
- Add below policy to the SageMaker Studio Role
  - Add IAM policy as in path inference-resources/data_scientist_studio_policy.json
- Go to EKS cluster and Select Access 
   - Select "Create Access Entry"
   - Select the SageMaker Studio Role and attach the policy AmazonEKSClusterAdminPolicy


## 10.0 Uninstallation

In [6]:
!helm uninstall hyperpod-inference-operator -n kube-system

Error: Kubernetes cluster unreachable: the server has asked for the client to provide credentials


### ENDPOINT CREATION

### L1 and intelligent routing enabled

To 